In [2]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [28]:
start_date = '2022-10-01'
end_date = '2022-10-31'

In [29]:
auto = f"""

WITH base_month_customer AS 
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d') AS month,
    customer_id,
    --MAX(auto_lifetime_last_ride_date) 
    auto_lifetime_last_ride_date,
    auto_lifetime_first_ride_date,
    --MAX(auto_lifetime_rides) 
    auto_lifetime_rides

FROM datasets.iallocator_customer_segments
WHERE 
    run_date BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date)) , '%Y-%m-%d') 
    AND run_date = DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d')
    AND (auto_lifetime_rides > 0 or auto_lifetime_rides is not null or auto_lifetime_last_ride_date is not null)

    AND auto_lifetime_rides BETWEEN 1 AND 4
--GROUP BY 1,2
),

base_month_segment AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    customer_id,
    CASE
    WHEN auto_lifetime_rides IS NULL OR DATE(month) = DATE(auto_lifetime_first_ride_date) THEN 'LTR =0'
    WHEN auto_lifetime_rides BETWEEN 1 AND 4 AND auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) THEN 'LTR ' || cast(auto_lifetime_rides as varchar)
    WHEN auto_lifetime_rides BETWEEN 1 AND 4 THEN 'LTR 1-4'
    WHEN (auto_lifetime_rides BETWEEN 5 AND 20) AND (auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN 'LTR 5-20'
    WHEN (auto_lifetime_rides BETWEEN 21 AND 50) AND (auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN 'LTR 21-50'
    WHEN (auto_lifetime_rides > 50) AND (auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN 'LTR 50+'
    WHEN auto_lifetime_rides >= 5 THEN 'LTR >=5'
    ELSE 'LTR =0'
    END ltr_auto,
    CASE 
    WHEN auto_lifetime_last_ride_date IS NULL OR DATE(month) = DATE(auto_lifetime_first_ride_date) THEN 'LTR=0'
    
    WHEN auto_lifetime_rides BETWEEN 1 AND 4 and auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '7' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '0' DAY) THEN '1. 1_7_DAYS'
    WHEN auto_lifetime_rides BETWEEN 1 AND 4 and auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '14' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '8' DAY) THEN '1. 8_14_DAYS'
    WHEN auto_lifetime_rides BETWEEN 1 AND 4 and auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '21' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '15' DAY) THEN '1. 15_21_DAYS'
    WHEN auto_lifetime_rides BETWEEN 1 AND 4 and auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '22' DAY) THEN '1. 22_30_DAYS'
    
    WHEN auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) THEN '1. LAST_30_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '45' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '2. 31_45_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '60' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '46' DAY) THEN '3. 46_60_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '61' DAY) THEN '4. 61_90_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '180' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '91' DAY) THEN '5. 91_180_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '181' DAY) THEN '6. 181_365_DAYS'
    WHEN auto_lifetime_last_ride_date < (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) THEN '7. 365_ABOVE'
    ELSE 'LTR=0'
    END recency_auto
FROM base_month_customer
WHERE auto_lifetime_rides BETWEEN 1 AND 4
),

base_month_total AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    COUNT(DISTINCT customer_id) existing_custr_total 
FROM base_month_customer
GROUP BY 1
),

customer_rf_daily_kpi AS
(
SELECT 
    day,
    customerid,
    rr_sessions_unique_daily,
    net_rides_daily,
    ao_sessions_unique_daily,
    fe_sessions_unique_daily,
    discount_daily,
    subtotal_daily
        
FROM datasets.customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
    AND service_name = 'Auto'
    AND customerid IS NOT NULL
),

rr_net AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customerid as customer_id,
    SUM(rr_sessions_unique_daily) rr,
    SUM(net_rides_daily) net_rides,
    SUM(discount_daily) burn,
    SUM(subtotal_daily) subtotal
FROM customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
GROUP BY 1,2
),

max_daily_ao AS
(
SELECT 
        day,
        customer_id,
        SUM(ao) AS ao,
        SUM(fe) AS fe
FROM
    (
    SELECT 
        customerid as customer_id,
        day,
        max(ao_sessions_unique_daily) AS ao,
        max(fe_sessions_unique_daily) AS fe
    FROM datasets.customer_rf_daily_kpi
    WHERE 
        day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
        AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
        AND customerid IS NOT NULL
    GROUP BY 1,2
    )
GROUP BY 1,2
),

monthly_ao AS
(
SELECT  
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customer_id,
    SUM(ao) AS ao,
    SUM(fe) AS fe
FROM max_daily_ao
GROUP BY 1,2
),

base_current AS (
SELECT 
        COALESCE(b.month,c.month,r.month) month,
        -- b.month,
        COALESCE(b.customer_id,c.customer_id,r.customer_id) customers,
        b.customer_id bcustomer_id,
        COALESCE(b.ltr_auto,'NEW') ltr_auto,
        COALESCE(b.recency_auto, 'NEW') recency_auto,
        c.customer_id acustomer_id,
        r.customer_id rcustomer_id,
        CASE 
        WHEN c.ao = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN c.ao = 0 AND c.fe > 0 THEN c.fe
        WHEN c.ao = 0 AND r.rr > 0 THEN r.rr
        WHEN c.ao IS NULL THEN 0
        ELSE c.ao END ao,
        
        CASE
        WHEN c.fe = 0 AND r.net_rides > 0 THEN r.net_rides 
        WHEN c.fe = 0 AND r.rr > 0 THEN r.rr
        WHEN c.fe IS NULL THEN 0
        ELSE c.fe END fe,
        
        CASE
        WHEN r.rr = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN r.rr IS NULL THEN 0
        ELSE r.rr END rr,
        
        COALESCE(r.net_rides, 0) net_rides,
        COALESCE(r.burn, 0) burn,
        COALESCE(r.subtotal, 0) subtotal

FROM base_month_segment AS b
FULL JOIN monthly_ao AS c ON b.customer_id = c.customer_id AND b.month = c.month 
FULL JOIN rr_net AS r ON b.customer_id = r.customer_id AND c.month = r.month
),

detailed_view AS 
(
SELECT 
        c.month,
        ltr_auto,
        case 
        when ltr_auto in ('LTR 1', 'LTR 2', 'LTR 3', 'LTR 4') then 1
        when ltr_auto = 'LTR 1-4' then 2
        when ltr_auto = 'LTR 5-20' then 3
        when ltr_auto = 'LTR 21-50' then 4
        when ltr_auto = 'LTR 50+' then 5
        when ltr_auto = 'LTR >=5' then 6
        when ltr_auto = 'LTR =0' then 7
        when ltr_auto = 'NEW' then 8
        end row_order,
        recency_auto,
        COUNT(DISTINCT bcustomer_id) existing_custr_count,
        COUNT(DISTINCT CASE WHEN ao > 0 THEN acustomer_id END) ao_users,
        COUNT(DISTINCT CASE WHEN fe > 0 THEN acustomer_id END) fe_users,
        COUNT(DISTINCT CASE WHEN rr > 0 THEN rcustomer_id END) rr_users,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN rcustomer_id END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
        
FROM base_current c
GROUP BY 1,2,3,4
),

summary_view AS (
SELECT 
        c.month,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN c.customers END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
FROM base_current c
GROUP BY 1
),

a as 
(
SELECT  
        a.month,
        a.row_order,
        a.ltr_auto,
        a.recency_auto,
        a.existing_custr_count,
        COALESCE(try(a.existing_custr_count*100.00/b.existing_custr_total), 0) existing_custr_distr,
        a.ao_users,
        COALESCE(try(a.ao_users*100.00/a.existing_custr_count), 0) ao_conversion, 
        a.fe_users,
        COALESCE(try(a.fe_users*100.00/a.existing_custr_count), 0) fe_conversion, 
        a.rr_users,
        COALESCE(try(a.rr_users*100.00/a.existing_custr_count), 0) rr_conversion,
        COALESCE(try(a.rr_users*100.00/a.fe_users), 0) fe_rr,
        a.net_users,
        COALESCE(try(a.net_users*100.00/a.existing_custr_count), 0) net_conversion,
        COALESCE(try(a.net_users*100.00/a.rr_users), 0) rr_net,
        COALESCE(try(a.net_users*100.00/c.net_users), 0) net_users_distr,
        a.net_rides,
        COALESCE(try(a.net_rides*100.00/c.net_rides), 0) net_rides_distr,
        COALESCE(try(a.net_rides*1.00/a.net_users), 0) rpc,
        a.discount,
        a.subtotal
        
FROM detailed_view a
LEFT JOIN base_month_total b ON a.month = b.month
LEFT JOIN summary_view c ON a.month = c.month
ORDER BY 1,2,3
),

b as 
(
SELECT
        month,
        8 row_order,
        'TOTAL' ltr_auto,
        '' recency_auto,
        SUM(existing_custr_count) existing_custr_count,
        100.00 existing_custr_distr,
        SUM(ao_users) ao_users,
        COALESCE(try(SUM(ao_users)*100.00/SUM(existing_custr_count)), 0) ao_conversion,
        SUM(fe_users) fe_users,
        COALESCE(try(SUM(fe_users)*100.00/SUM(existing_custr_count)), 0) fe_conversion,
        SUM(rr_users) rr_users,
        COALESCE(try(SUM(rr_users)*100.00/SUM(existing_custr_count)), 0) rr_conversion,
        COALESCE(try(SUM(rr_users)*100.00/SUM(fe_users)), 0) fe_rr,
        SUM(net_users) net_users,
        COALESCE(try(SUM(net_users)*100.00/SUM(existing_custr_count)), 0) net_conversion,
        COALESCE(try(SUM(net_users)*100.00/SUM(rr_users)), 0) rr_net,
        100.00 net_users_distr,
        SUM(net_rides) net_rides,
        100.00 net_rides_distr,
        COALESCE(try(SUM(net_rides)*1.0/SUM(net_users)), 0) rpc,
        SUM(discount) discount,
        SUM(subtotal) subtotal
FROM a
GROUP BY 1
),

final as 
(
select * from a
union
select * from b
)
select 
        month,
        ltr_auto,recency_auto,
        existing_custr_count,existing_custr_distr,
        ao_users,ao_conversion,
        fe_users,fe_conversion,
        rr_users,rr_conversion,fe_rr "fe_rr(users)",
        net_users,net_conversion, rr_net "rr_net(users)", net_users_distr,
        net_rides,net_rides_distr,
        rpc,
        discount,
        subtotal,
        COALESCE(TRY(discount*100.0/subtotal),0) discount_perc,
        COALESCE(TRY(discount*1.0/net_rides),0) dpr
from final
order by 1,row_order,3

"""

In [30]:
df_auto = pd.read_sql(auto, conn)
df_auto

,month,ltr_auto,recency_auto,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-10-01,LTR 1,1. 15_21_DAYS,129501,1.73,82056,63.36,73184,56.51,39008,...,16.94,56.22,0.73,38321,0.51,1.75,125949.0,5.311239e+06,2.371368,3.286684
1,2022-10-01,LTR 4,1. 15_21_DAYS,41198,0.55,31871,77.36,30122,73.12,20302,...,31.60,64.12,0.44,25488,0.34,1.96,63274.0,3.507398e+06,1.804015,2.482502
2,2022-10-01,LTR 3,1. 15_21_DAYS,59514,0.79,40495,68.04,37899,63.68,24544,...,25.83,62.62,0.52,29244,0.39,1.90,75592.0,3.984902e+06,1.896960,2.584872
3,2022-10-01,LTR 2,1. 15_21_DAYS,75504,1.01,53792,71.24,49775,65.92,30189,...,24.09,60.25,0.61,33336,0.44,1.83,97170.0,4.573761e+06,2.124510,2.914867
4,2022-10-01,LTR 2,1. 1_7_DAYS,117965,1.57,100097,84.85,95267,80.76,71557,...,45.34,74.75,1.79,117569,1.55,2.20,294194.0,1.592802e+07,1.847022,2.502309
5,2022-10-01,LTR 1,1. 1_7_DAYS,152126,2.03,115336,75.82,105019,69.03,63704,...,25.64,61.22,1.31,81504,1.08,2.09,220796.0,1.104792e+07,1.998531,2.709020
6,2022-10-01,LTR 4,1. 1_7_DAYS,76166,1.02,67802,89.02,65719,86.28,53441,...,55.06,78.47,1.41,108690,1.44,2.59,236691.0,1.456212e+07,1.625388,2.177670
7,2022-10-01,LTR 3,1. 1_7_DAYS,94781,1.26,81002,85.46,77923,82.21,61617,...,50.14,77.13,1.59,115216,1.52,2.42,258743.0,1.553047e+07,1.666035,2.245721
8,2022-10-01,LTR 1,1. 22_30_DAYS,158416,2.11,92942,58.67,82498,52.08,42597,...,14.64,54.46,0.78,39468,0.52,1.70,128593.0,5.518922e+06,2.330038,3.258159
9,2022-10-01,LTR 3,1. 22_30_DAYS,72179,0.96,44186,61.22,41094,56.93,25545,...,21.49,60.72,0.52,28145,0.37,1.81,70328.0,3.864284e+06,1.819949,2.498774


In [31]:
df_auto.to_csv("oct-auto-ltr_1-4.csv", index = False)

In [21]:
df_auto.to_csv("nov-auto-ltr_1-4.csv", index = False)